# 債券報價中的 YTM、Spot Rate、Forward Rate

## 輸入資料：債券現值、面值、年期、配息、付息次數

In [17]:
pre_value = float(input("Input the bond's present value:(請輸入債券現值):"))  #債券現值、價格

Input the bond's present value:(請輸入債券現值):102.91


In [2]:
par_value = float(input("Input the bond's par value:(請輸入債券面值):"))  #債券票面價值

Input the bond's par value:(請輸入債券面值):100


In [3]:
year = float(input("Input the year of the bond:(請輸入債券的年期):"))  #年期數(固定1.5或2)

Input the year of the bond:(請輸入債券的年期):1.5


In [18]:
bond_coup = float(input("Input the coupon rate(%) of the bond :(請輸入債息百分比):"))  #債券配息率

Input the coupon rate(%) of the bond :(請輸入債息百分比):5


In [5]:
freq = int(input("Input the times of interest payments per year(1 or 2):(每年付息次數(1或2)):"))  #每年付息次數

Input the frequency of interest payments per year(1 or 2):(每年付息次數(1或2)):2


## 計算債券的到期收益率(YTM)

In [19]:
#判斷是否為正實數的函數(後面選值須用到)
def is_pos_num(s):
    try:
        int(s)
    except:
        return False
    if s<=0:
        return False
    return True

In [27]:
#YTM函數
from sympy import *
def YTM_of_bond(pre, par, year, coup, freq):
    if coup == 0:
        periods = year*freq
        return ((par/pre)**(1/periods)-1)*freq
    else:
        periods = int(year*freq)
        per_coup = coup/100*par/freq

        y = symbols('y')
        func = sum([per_coup/(1+y/freq)**(i+1) for i in range(periods)]) + par/(1+y/freq)**periods - pre  #方程式
        ytm = [x for x in solve(func,y) if is_pos_num(x)][0]  #只有一個，取第一個就是解
        #return solve(func,y)  #這裡會有負數與複數
        return ytm

In [21]:
YTM_of_bond(pre_value, par_value, year, bond_coup, freq)  #該債券的YTM

0.0300148213380233

## 計算即期利率(Spot Rate)

In [37]:
#給定一個半年、一個一年的 Zero Coupon Bond的資料(計算1.5年的需要)：
price_of_B1 = 99.5
price_of_B2 = 98.03

YTM_of_bond(price_of_B1, 100, 0.5, 0, 2),YTM_of_bond(price_of_B2, 100, 1, 0, 2) #上面兩個 bond的 YTM

(0.010050251256281229, 0.019995929712302463)

In [24]:
#Spot Rate函數
def SR_of_bond(pre, par, year, coup, freq):
    periods = int(year*freq)
    per_coup = coup/100*par/freq
    s = Symbol('s')   
    
    if coup == 0:
        return YTM_of_bond(pre, par, year, coup, freq)    #零息債 SR=YTM
    
    elif freq == 1:
        S_R = [YTM_of_bond(price_of_B2, 100, 1, 0, freq)]  #需看一年的
        for n in range(periods-1):
            func = sum([per_coup/(1+S_R[i])**(i+1) for i in range(n+1)]) + (per_coup+par)/(1+s)**(n+2) - pre
            S_R.append([s for s in solve(func,s) if is_pos_num(s)][0])

        return S_R[-1]
    
    elif freq == 2:
        S_R = [YTM_of_bond(price_of_B1, 100, 0.5, 0, freq),YTM_of_bond(price_of_B2, 100, 1, 0, freq)]  #需看半年與一年的
        for n in range(periods-2):
            func = sum([per_coup/(1+S_R[i]/2)**(i+1) for i in range(n+2)]) + (per_coup+par)/(1+s/2)**(n+3) - pre
            S_R.append([s for s in solve(func,s) if is_pos_num(s)][0])
          
        return S_R[-1]
        

In [28]:
SR_of_bond(price_of_B1, 100, 0.5, 0, freq) #B1的SR

0.010050251256281229

In [31]:
SR_of_bond(price_of_B2, 100, 1.0, 0, freq) #B2的SR

0.019995929712302463

In [33]:
SR_of_bond(pre_value, par_value, year, bond_coup, freq) #new bond's SR

0.0303504009904628

## 計算遠期利率(Forward Rate)並建立對照表

In [34]:
#先建好資料
pre_list = [price_of_B1, price_of_B2, pre_value]
par_list = [100, 100, par_value]
coup_list = [0, 0 , bond_coup]

import numpy as np
def FR_of_bond(pre, par, year, coup, freq):
    periods = int(year*freq)
    data = np.array([[None]*periods]*periods)  #建立 nxn array
    for i in range(periods):
        for j in range(periods):
            if j>i:  #對角線以上才有值
                data[i][j] = ((1+SR_of_bond(pre_list[j], par_list[j], (j+1)/freq, coup_list[j], freq))**(j+1)
                              /(1+SR_of_bond(pre_list[i], par_list[i], (i+1)/freq, coup_list[i], freq))**(i+1))**(1/(j-i))-1
    return data

In [35]:
FR_of_bond(pre_value, par_value, year, bond_coup, freq)

array([[None, 0.030039540444294577, 0.0406529655420242],
       [None, None, 0.0513757503175936],
       [None, None, None]], dtype=object)

In [36]:
#Forward Rate表
import pandas as pd
periods = int(year*freq)
df = pd.DataFrame(FR_of_bond(pre_value, par_value, year, bond_coup, freq))
df

,0,1,2
0,None,0.0300395,0.0406529655420242
1,None,None,0.0513757503175936
2,None,None,None
